<a href="https://colab.research.google.com/github/Sushil211/CNN/blob/master/simple_cnn_on_4classimages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
! cp kaggle.json /root/.kaggle/kaggle.json

In [5]:
! kaggle

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: too few arguments


In [6]:
! chmod 600 /root/.kaggle/kaggle.json #Giving Read write option only to the owner

In [7]:
! kaggle datasets download ellenyusa/4classimages

 86% 33.0M/38.5M [00:01<00:00, 16.5MB/s]
100% 38.5M/38.5M [00:01<00:00, 33.2MB/s]


In [8]:
! unzip -qq 4classimages.zip

In [9]:
! rm -rf data/data

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
gen = ImageDataGenerator(rotation_range=15, shear_range=.1, zoom_range=.3)

generator = gen.flow_from_directory("data/")

Found 808 images belonging to 4 classes.


In [12]:
generator.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}

In [13]:
X, y = next(generator)
X.shape, y.shape

((32, 256, 256, 3), (32, 4))

In [14]:
y #It is already converted to y_hot 

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [15]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model



in_layer = Input(shape=(256, 256, 3))
conv = Conv2D(filters=60, kernel_size=3)(in_layer)
pooling = MaxPooling2D()(conv) 
conv = Conv2D(filters=60, kernel_size=3)(pooling)
pooling = MaxPooling2D()(conv) 
conv = Conv2D(filters=60, kernel_size=3)(pooling)
pooling = MaxPooling2D()(conv) 
flat = Flatten()(pooling)
d1 = Dense(units=400, activation="tanh")(flat)
d2 = Dense(units=100, activation="tanh")(d1)
out_layer = Dense(units=4, activation="softmax")(d2)

model = Model(in_layer, out_layer)

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 60)      1680      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 60)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 60)      32460     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 60)        32460     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 60)        0     

In [16]:
model.fit_generator(generator, steps_per_epoch=20, epochs=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
20/20 [==============================] - 11s 530ms/step - loss: 1.6049 - accuracy: 0.2656
Epoch 2/5
20/20 [==============================] - 11s 535ms/step - loss: 1.4198 - accuracy: 0.2175
Epoch 3/5
20/20 [==============================] - 11s 527ms/step - loss: 1.4022 - accuracy: 0.2305
Epoch 4/5
20/20 [==============================] - 11s 532ms/step - loss: 1.4051 - accuracy: 0.2321
Epoch 5/5
20/20 [==============================] - 11s 529ms/step - loss: 1.4066 - accuracy: 0.1997


<b>Observe that accuracy is less than 20% for having only 4 class data. So simple cnn is unable to perform well in this type of dataset, hence we must use transfer learning to make our cnn model better and increase our accuracy.</b>